In [1]:
#import all necessary libraries
import json
from ExchangeRateEvent import *
from get_currencies import *
from constants import API_URL, BASE_CURRENCY, LOCAL_CURRENCIES, SALESFORCE_INSTANCE_URL, SALESFORCE_API_USER, SALESFORCE_API_PASS, SALESFORCE_API_TOKEN, DB_URL
import logging
import os
import sys
from datetime import timedelta, date
import datetime
import pandas as pd
from file_opener import *
from simple_salesforce import Salesforce
from dateutil.parser import parse

In [2]:
#Navigate and import from the submodule
sys.path.append('..\..')
from data_utils.sf_connect import sf_connect
from data_utils.file_opener import file_opener

<>:2: SyntaxWarning: invalid escape sequence '\.'
<>:2: SyntaxWarning: invalid escape sequence '\.'
/var/folders/gz/65b9jhcd3hsds0mhwvdrrbbc0000gn/T/ipykernel_7398/2499133269.py:2: SyntaxWarning: invalid escape sequence '\.'
  sys.path.append('..\..')


In [3]:
#set up parameters
API_KEY = os.environ['API_KEY']
file_date = datetime.datetime.now().strftime('%Y%m%d')

Run Query using Salesforce API
Alternatively, you could download the results of the SOQL query from Data Export in Salesforce

In [5]:
#load in SOQL query used to pull all relevant donations
donation_soql = file_opener('donations.soql')

In [6]:
#connect to the Salesforce API using the sf_connect.py file in the data_utils submodule
sf = sf_connect()

In [7]:
#run the SOQL query using the API
sf.sf_query_object(donation_soql)

Querying data from Salesforce for the opportunity object...
Query completed.
Parsing query results...
Converting results to dataframe...
Converting data types...
Converting column Local_Currency_Amount_Number_For_Nat__c to float64...
Converting column CloseDate to date...


In [9]:
#save results of query as a local dataframe
sf_donations_df = sf.data

In [8]:
#set up a mapping to rename the Salesforce fields locally to match CurrencyAPI field names;
renamer = {'Id': 'opportunity_id'
           ,'Currency__c': 'local_currency'
           ,'Local_Currency_Amount_Number_For_Nat__c': 'local_currency_amount'
           ,'CloseDate': 'exchange_rate_date'}

In [10]:
#apply the field-renaming mapping to the dataframe, and save a list of unique donation dates locally (to help pull the minimum amount of data from the CurrencyAPI)
if len(sf_donations_df) > 0:
    sf_donations_df.rename(columns = renamer, inplace = True)
    currencies_dates = sf_donations_df[['local_currency', 'exchange_rate_date']].copy().drop_duplicates()

In [11]:
#print how many donations will need converting:
print('The Salesforce query returned a dataframe with {} records.'.format(len(sf_donations_df)))

The Salesforce returned a dataframe with 1297 records.


In [12]:
#get the exchange rate for each donation from the Currency API using the get_currencies.py file

if len(currencies_dates) > 0:
    records = []
    for c in currencies_dates.iterrows():
        data = c[1]
        local_currency = data[0]
        date = data[1].strftime('%Y%m%d')

        exchange_rate = get_currencies(
                                       BASE_CURRENCY
                                       ,local_currency
                                       ,date
                                       ,API_KEY
                                       ,API_URL
                                       )
        records.append(exchange_rate)

/var/folders/gz/65b9jhcd3hsds0mhwvdrrbbc0000gn/T/ipykernel_7398/3284135293.py:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  local_currency = data[0]
/var/folders/gz/65b9jhcd3hsds0mhwvdrrbbc0000gn/T/ipykernel_7398/3284135293.py:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  date = data[1].strftime('%Y%m%d')
INFO:root:Successful request to the exchange rate API endpoint.
INFO:root:After this request, 14999 of 15000 requests have been made against our quota for a utilization percentage of 6.666666666665932e-05
INFO:root:Successful request to the exchange rate API endpoint.
INFO:root:After this request, 14998 of 150

In [13]:
#save results
data = [rows.__dict__ for days in records for rows in days]

In [14]:
#format results as a dataframe
if len(data) > 0:
    all_exchange_rates = pd.DataFrame(data)
    all_exchange_rates['exchange_rate_date'] = all_exchange_rates.apply(lambda x: parse(x['exchange_rate_date']).date(), axis = 1)

In [15]:
#save results as a CSV
if len(all_exchange_rates) > 0 and len(sf_donations_df) > 0:
    conversion = sf_donations_df.merge(all_exchange_rates[['local_currency', 'exchange_rate_date', 'exchange_rate']]
                                       ,how = 'left'
                                       ,on = ['local_currency', 'exchange_rate_date']
                                       ,indicator = True)
    
    conversion['base_currency_amount'] = conversion.apply(lambda x: round(x['local_currency_amount']/x['exchange_rate'], 2), axis = 1)
    
    print('{} records were successfuly converted.'.format(len(conversion[conversion['_merge'] == 'both'])))
    
    conversion[conversion['_merge'] == 'both'].to_csv('currency_conversion_{}.csv'.format(file_date))
    
    #conversion[conversion['_merge'] != 'both'].to_csv('no_currency_conversion_{}.csv'.format(file_date))

1297 records were successfuly converted.
